# Wizards Staff Tutorial for Calcium Imaging Analysis

<img src="../img/wizards_staff.png" alt="Wizards Staff logo" width="400"/>

---

## Welcome Young Wizard

This tutorial will guide you through analyzing your calcium imaging data using **Wizards Staff**. By the end, you'll be able to:

- Load your processed calcium imaging data
- Extract meaningful metrics about neuronal activity
- Identify which neurons fire together (network analysis)
- Visualize your results
- Export data for statistical analysis

---

## What is Wizards Staff?

**Wizards Staff** is a Python toolkit that takes the *processed* output from calcium imaging pipelines (like [Lizard-Wizard](https://github.com/ArcInstitute/Lizard-Wizard)) and extracts biologically meaningful metrics.

### Biological Questions Wizards Staff Helps Answer:

| Question | Wizards Staff Output |
|----------|---------------------|
| How active are my neurons? | **Firing rate** (spikes per minute) |
| How quickly do neurons respond? | **Rise time** (time to peak) |
| How long do calcium events last? | **FWHM** (event duration) |
| Which neurons fire together? | **Pairwise correlations** |
| Are there functional groups? | **K-means clustering** |
| How do conditions compare? | **Group comparisons** |

---

## Prerequisites

Before starting, you need:

1. **Completed Lizard-Wizard run** — Your calcium imaging videos must already be processed
2. **Output folder** — Know where your Lizard-Wizard results are saved
3. **Metadata file** — A simple CSV file describing your samples (we'll show you how to make one)



---

# 📚 Notebook Structure

This tutorial is organized into **four parts**. Depending on your experimental design, you may not need all of them:

| Part | Sections | Who Should Read |
|------|----------|-----------------|
| **Part 1: Shared Setup** | 1-4 | Everyone |
| **Part 2: Unpaired Group Comparisons** | 5-10 | Different samples per condition (Control vs Treated) |
| **Part 3: Paired Drug Response** | 11-16 | Same samples before/after treatment |
| **Part 4: Advanced & Reference** | 17-19 | Optional - power users and troubleshooting |

**Quick Start:** Complete Part 1, then follow the decision guide in Section 4 to choose Part 2 or Part 3.

---

# Part 1: Shared Setup

*Complete this section regardless of your experimental design.*

---

# Section 1: Understanding Your Input Files

**Learning objective**: Know what files Wizards Staff needs and where to find them.

---

## Files from Lizard-Wizard

When Lizard-Wizard finishes processing your videos, it creates several files for each sample. Here's what each file contains:

| File Pattern | What It Contains | Plain English |
|--------------|-----------------|---------------|
| `*_dff-dat.npy` | ΔF/F₀ traces | The normalized calcium activity for each neuron over time |
| `*_cnm-A.npy` | Spatial footprints | Where each neuron is located in the image |
| `*_cnm-idx.npy` | Accepted components | Which detected "cells" passed quality control |
| `*_minprojection.tif` | Reference image | A static image showing your field of view |
| `*_masks.tif` | ROI masks | Outlines of each neuron (optional, for shape analysis) |
| `*_f-dat.npy` | Raw fluorescence | The original brightness values before normalization |

💡 **Tip**: The `*` represents your sample name. For example, if your sample is called `Experiment1_WellA01`, the files would be `Experiment1_WellA01_dff-dat.npy`, etc.

### Finding Your Results Folder

Your results folder is the `--output_dir` you specified when running Lizard-Wizard. It typically looks something like:

```
/path/to/lizard-wizard-output/
├── Sample1_dff-dat.npy
├── Sample1_cnm-A.npy
├── Sample1_cnm-idx.npy
├── Sample1_minprojection.tif
├── Sample2_dff-dat.npy
├── Sample2_cnm-A.npy
...
```

In [ ]:
# Example: View sample metadata structure
import pandas as pd

metadata_example = pd.DataFrame({
    'Sample': ['Exp1_DMSO_A01_s1', 'Exp1_DMSO_A02_s1', 'Exp1_Drug_B01_s1', 'Exp1_Drug_B02_s1'],
    'Well': ['Control', 'Control', 'Treated', 'Treated'],
    'Frate': [30, 30, 30, 30]
})

print("Example metadata format:")
display(metadata_example)

## Creating Your Metadata File

The metadata file tells Wizards Staff about your experimental design. It's a simple CSV (spreadsheet) with **three required columns**:

| Column | Description | Example |
|--------|-------------|--------|
| `Sample` | Unique name matching your filenames | `Experiment1_WellA01` |
| `Well` | Group/condition identifier | `Control`, `Treated`, `A01`, etc. |
| `Frate` | Recording frame rate (frames per second) | `30` |

### Example Metadata File

Here's what your `metadata.csv` might look like:

```csv
Sample,Well,Frate
Exp1_DMSO_A01_s1,Control,30
Exp1_DMSO_A02_s1,Control,30
Exp1_Drug_B01_s1,Treated,30
Exp1_Drug_B02_s1,Treated,30
```

⚠️ **Important**: The `Sample` column must **exactly match** the prefix of your filenames (everything before `_dff-dat.npy`).

**Tip**: You can create this file in Excel and save it as CSV, or use any text editor.

In [ ]:
# ╔════════════════════════════════════════════════════════════════════════════╗
# ║  SECTION 2: SETUP AND INSTALLATION                                          ║
# ╚════════════════════════════════════════════════════════════════════════════╝

# INSTALLATION (only need to run this once)
# -----------------------------------------
# Remove the '#' from the line below and run this cell if you haven't installed Wizards Staff yet.
# After installation, you can add the '#' back to skip this step in future sessions.

# !pip install git+https://github.com/ArcInstitute/Wizards-Staff.git

---

# Section 2: Setup and Installation

**Learning objective**: Install Wizards Staff and import it into Python.

---

Run the cell above to install (if needed), then run the cell below to import the main tools from our package:

In [ ]:
# IMPORT THE MAIN TOOLS
# ---------------------
# These are the Python packages we'll use throughout this tutorial.

# The main Wizards Staff class - this is the "container" for all your data
from wizards_staff import Orb

# Standard tools for viewing data (optional but helpful)
import matplotlib.pyplot as plt  # For viewing plots

# Make plots appear directly in the notebook
%matplotlib inline

print("All packages loaded successfully!")

### What is an "Orb"?

Think of the **Orb** as your **crystal ball** that:
- Organizes all your samples in one place
- Keeps track of which files belong to which sample
- Provides all the analysis methods you need
- Stores your results for easy access

You'll create one Orb, and it will handle everything from there.

---

# Section 3: Loading Your Data

**Learning objective**: Create an Orb and load your calcium imaging data.

---

## Step 3.1: Set Your File Paths

You need to tell Wizards Staff where to find your data. **Edit the paths below** to match your own files:

In [ ]:
# ╔════════════════════════════════════════════════════════════════════════════╗
# ║  ⚠️ EDIT THESE PATHS TO MATCH YOUR DATA  ⚠️                                ║
# ╚════════════════════════════════════════════════════════════════════════════╝

# Path to your Lizard-Wizard output folder
# This is the folder containing all the *_dff-dat.npy, *_cnm-A.npy, etc. files
RESULTS_FOLDER = "/path/to/your/lizard-wizard-output/"
RESULTS_FOLDER = "/large_storage/cmtc/public/CMTC-Sph10162025-plateB-week5"
# Path to your metadata CSV file
# This is the spreadsheet you created with Sample, Well, and Frate columns
METADATA_FILE = "/path/to/your/metadata.csv"
METADATA_FILE = "/large_storage/cmtc/public/CMTC-Sph10162025-plateB-week5/metadata.csv"
# Where to save your analysis results
# A new folder will be created if it doesn't exist
OUTPUT_FOLDER = "./my_analysis_results/"

## Step 3.2: Create the Orb

Now we'll create an Orb with your data. This step:
1. Reads your metadata file
2. Scans for all matching data files
3. Validates that files exist for each sample
4. Reports any missing files

In [ ]:
# CREATE THE ORB
# --------------
# This creates your "crystal ball" and loads information about all your samples.

orb = Orb(
    results_folder=RESULTS_FOLDER,      # Where your data files are
    metadata_file_path=METADATA_FILE    # Your experimental design info
)

# Show what was found
print(f"\n Found {len(orb.samples)} samples in your metadata")
print(f"\n Sample names:")
for sample in sorted(orb.samples):
    print(f"   • {sample}")

### What to Look For

 **Success**: You should see a list of your sample names with no errors.

⚠️ **Warnings about missing files**: Some samples may be missing certain files. Common causes:
- Typos in the `Sample` column of your metadata
- Files in a different folder
- Lizard-Wizard didn't finish processing some samples

**Tip**: If you see warnings, double-check that your `Sample` names exactly match the file prefixes.

## Step 3.3: Check What Files Were Found (Optional)

You can verify which data types are available for your samples:

In [ ]:
# CHECK AVAILABLE DATA TYPES
# --------------------------
# This shows which types of files were found across all samples.

if orb.input_files is not None:
    available_data = set(orb.input_files['DataItem'])
    print("Available data types:")
    for data_type in sorted(available_data):
        print(f"   ✓ {data_type}")
else:
    print("❌ No input files found. Please check your RESULTS_FOLDER path.")

---

# Section 4: Choose Your Analysis Workflow

Determine which analysis track matches your experimental design.

---

Now that your data is loaded, the next step depends on your **experimental design**. Wizards Staff supports two fundamentally different workflows:

## Which Workflow Matches Your Experiment?

| If your experiment has... | Design Type | Statistical Approach | Go To |
|---------------------------|-------------|---------------------|-------|
| **Different samples** in Control vs Treatment groups | Unpaired | Independent samples tests | **Part 2 (Section 5)** |
| **Same samples** imaged before AND after treatment | Paired | Paired tests, fold-change | **Part 3 (Section 11)** |

---

## Examples of Each Design

### 🔵 Unpaired Design (Part 2)

Use Part 2 if you have **different samples** in each experimental condition:

- Comparing firing rates between **wild-type and knockout** organoids
- Testing **multiple drug concentrations** on separate wells  
- **Control group on Plate 1**, Treated group on Plate 2
- Comparing **different cell lines** or **genotypes**

**Statistical approach**: Independent t-test, Mann-Whitney U, ANOVA

➡️ **Continue to Section 5** (you're already in the right place!)

---

### 🟢 Paired Design (Part 3)

Use Part 3 if you image the **same samples at two timepoints**:

- Baseline recording → **apply drug** → record same organoid again
- Pre-treatment → **drug wash-in** → post-treatment  
- Same wells imaged at **Day 0 and Day 7**
- **Washout experiments**: baseline → drug → wash → recovery

**Statistical approach**: Paired t-test, fold-change normalization

➡️ **Skip Ahead to** Part 3: Paired Drug Response Analysis)

---

## Still Unsure?

Ask yourself: *"Do my Control and Treated data come from the same physical samples?"*

- **YES** → Part 3 (Paired)
- **NO** → Part 2 (Unpaired)

*Note: You can always come back and try the other workflow later. The Orb you've created works with both.*

---

# Part 2: Unpaired Group Comparisons

*For experiments with different samples in each condition.*

This part covers:
- Running the standard analysis pipeline
- Understanding neural activity metrics (firing rate, event kinetics, clustering)
- Correlation and network synchrony analysis
- Statistical comparisons between groups
- Creating publication-ready figures


---

# Section 5: Running the Analysis Pipeline

**Learning objective**: Run the complete analysis pipeline with one command.

---

## The `run_all` Method

The fastest way to analyze all your samples is using `run_all()`. This single command:

1.  **Filters** neurons to remove noise and artifacts
2.  **Detects** calcium transients (spikes) in each neuron
3.  **Calculates** rise times, peak durations, and firing rates
4.  **Clusters** neurons into functional groups
5.  **Computes** correlations between neurons
6.  **Generates** visualizations

---

## Understanding the Parameters

Before running, let's understand what each setting does:

### Essential Parameters (You Should Adjust These)

| Parameter | What It Does | How to Set It |
|-----------|--------------|---------------|
| `group_name` | Column for **comparing conditions** | Use `"Well"` or another column from your metadata |
| `show_plots` | **Display** plots in the notebook | `True` to preview, `False` for faster processing |
| `save_files` | **Save** results to disk | `True` when you're ready to export |
| `output_dir` | **Where** to save files | A folder path (will be created if needed) |

### Advanced Parameters (Usually Leave as Default)

| Parameter | What It Does | Default | When to Change |
|-----------|--------------|---------|----------------|
| `zscore_threshold` | How strong a signal must be to count as a "spike" | `3` | Lower (e.g., 2) if missing events; higher (e.g., 4) if too much noise |
| `p_th` | Brightness percentile for filtering dim neurons | `75` | Lower if losing too many neurons; higher if including artifacts |
| `size_threshold` | Maximum neuron size (pixels) | `20000` | Increase if neurons appear large; decrease to filter artifacts |
| `min_clusters` / `max_clusters` | Range for finding neuron groups | `2` / `10` | Adjust based on expected functional populations |

---

### What is a Z-score?

A **z-score** tells you how unusual a value is compared to the average:
- **Z = 0**: Average activity
- **Z = 1**: One standard deviation above average
- **Z = 3**: Three standard deviations above average (very unusual = likely a real spike)

With `zscore_threshold=3`, we only count events that are 3 standard deviations above the baseline — this helps distinguish real calcium transients from random noise.

## Step 4.1: Run the Analysis

In [ ]:
# ╔════════════════════════════════════════════════════════════════════════════╗
# ║  RUN THE COMPLETE ANALYSIS PIPELINE                                        ║
# ╚════════════════════════════════════════════════════════════════════════════╝

orb.run_all(
    # ── Essential settings (adjust these) ──────────────────────────────────────
    group_name="Well",         # Group by: Which metadata column defines your conditions?
    
    # ── Output settings ────────────────────────────────────────────────────────
    show_plots=False,           # Show plots: True = see plots now; False = faster
    save_files=False,           # Save results: True = export to files; False = just compute
    output_dir=OUTPUT_FOLDER,  # Where to save (uses OUTPUT_FOLDER from above)
    
    # ── Spike detection settings (usually leave as default) ───────────────────
    zscore_threshold=3,        # Spike sensitivity: Lower = more sensitive (2-5 typical)
    percentage_threshold=0.2,  # FWHM threshold: For measuring event duration
    
    # ── Neuron filtering settings (usually leave as default) ──────────────────
    p_th=75,                   # Brightness filter: Remove dim/noisy components (0-100)
    size_threshold=20000,      # Size filter: Remove artifacts larger than this (pixels)
    
    # ── Clustering settings (usually leave as default) ────────────────────────
    min_clusters=2,            # Minimum groups: Fewest functional clusters to try
    max_clusters=10,           # Maximum groups: Most functional clusters to try
    
    # ── Performance settings (adjust if kernel crashes) ────────────────────────
    threads=2,                 # Sequential processing: Use 1 to avoid memory issues
)

### What You Should See

During analysis, you'll see:
1. **Progress messages** as each sample is processed
2. **Activity maps** showing where neurons are located
3. **Cluster plots** showing functional neuron groups
4. **Trace plots** showing calcium activity over time
5. **Correlation plots** (if `group_name` was provided)

⚠️ **If you see warnings**: Some samples may be skipped due to missing files. This is normal if some recordings failed to be processed by Lizard Wizard.

**Tip**: For large datasets, set `show_plots=False` to speed up processing and reduce memory load

---

# Section 6: Understanding Your Metrics

**Learning objective**: Access and interpret the biological metrics from your analysis.

---

After running `run_all()`, your results are stored in the Orb as **DataFrames** (spreadsheet-like tables). Let's explore each one:

## 6.1: Firing Rate Data

### What It Measures
**Firing rate** = How many calcium transients ("spikes") each neuron produces per minute.

### Biological Interpretation
-  **Higher firing rate** → More active neurons
-  **Lower firing rate** → Less active or silent neurons
-  Useful for comparing overall activity levels between conditions (e.g., drug vs. control)

In [ ]:
# VIEW FIRING RATE DATA
# ---------------------
# This table shows how many spikes per minute each neuron produces.

if orb.frpm_data is not None:
    print("Firing Rate Data (spikes per minute)")
    print("="*50)
    print(f"Total neurons analyzed: {len(orb.frpm_data)}")
    print(f"\nFirst 10 rows:")
    display(orb.frpm_data.head(10))
    
    # Quick summary statistics
    print(f"\n Summary Statistics:")
    print(f"   Mean firing rate: {orb.frpm_data['Firing Rate Per Min'].mean():.2f} spikes/min")
    print(f"   Median firing rate: {orb.frpm_data['Firing Rate Per Min'].median():.2f} spikes/min")
    print(f"   Range: {orb.frpm_data['Firing Rate Per Min'].min():.2f} - {orb.frpm_data['Firing Rate Per Min'].max():.2f} spikes/min")
else:
    print("❌ No firing rate data available. Did run_all() complete successfully?")

## 6.2: Rise Time Data

### What It Measures
**Rise time** = How long it takes for a calcium signal to go from baseline to peak (in frames).


In [ ]:
# VIEW RISE TIME DATA
# -------------------
# This table shows how quickly each neuron's calcium signals rise to peak.

if orb.rise_time_data is not None:
    print("Rise Time Data (frames to peak)")
    print("="*50)
    print(f"\nFirst 10 rows:")
    display(orb.rise_time_data.head(10))
    
    # Note about interpretation
    print("\n To convert to seconds: divide by your frame rate")
    print(f"   Example: 10 frames at 30 fps = {10/30:.2f} seconds")
else:
    print("❌ No rise time data available.")

## 6.3: FWHM Data (Event Duration)

### What It Measures
**FWHM** (Full Width at Half Maximum) = How long a calcium event lasts, measured as the width of the peak at half its maximum height.


In [ ]:
# VIEW FWHM DATA
# --------------
# This table shows the duration of calcium events for each neuron.

if orb.fwhm_data is not None:
    print("FWHM Data (event duration in frames)")
    print("="*50)
    print(f"\nFirst 10 rows:")
    display(orb.fwhm_data.head(10))
else:
    print("❌ No FWHM data available.")

## 6.4: Clustering Data (Functional Groups)

### What It Measures
**Silhouette scores** = How well neurons separate into distinct functional groups based on their activity patterns.

### Biological Interpretation
- **Higher silhouette score** (closer to 1) → Neurons form clear, distinct groups
- **Lower silhouette score** (closer to 0) → Groups overlap; neurons have similar activity patterns
- Identifies co-active neuronal ensembles that may represent functional circuits

## 6.5: Fall Time Data (Signal Decay)

### What It Measures
**Fall time** = How long it takes for a calcium signal to return from peak to baseline (in frames).

### Biological Interpretation
- **Longer fall time** → Slower calcium clearance/buffering
- **Shorter fall time** → Faster signal termination
- Together with rise time, provides a complete picture of calcium event kinetics
- Can indicate differences in calcium handling machinery between neurons or conditions

In [ ]:
# VIEW FALL TIME DATA
# --------------------
# This table shows how long each neuron's calcium signals take to decay from peak.

if orb.fall_time_data is not None:
    print("Fall Time Data (frames from peak to baseline)")
    print("="*50)
    print(f"\nFirst 10 rows:")
    display(orb.fall_time_data.head(10))
    
    # Note about interpretation
    print("\n💡 To convert to seconds: divide by your frame rate")
    print(f"   Example: 15 frames at 30 fps = {15/30:.2f} seconds")
    
    # Compare rise vs fall times
    if orb.rise_time_data is not None:
        print("\n📊 Rise vs Fall Time Comparison:")
        rise_mean = orb.rise_time_data['Rise Times'].mean()
        fall_mean = orb.fall_time_data['Fall Times'].mean()
        print(f"   Mean rise time: {rise_mean:.2f} frames")
        print(f"   Mean fall time: {fall_mean:.2f} frames")
        print(f"   Ratio (fall/rise): {fall_mean/rise_mean:.2f}x")
else:
    print("❌ No fall time data available.")

## 6.6: Peak Amplitude Data (Transient Height)

### What It Measures
**Peak amplitude** = The height of each calcium transient, measured as ΔF/F₀ (change in fluorescence relative to baseline fluorescence).

### Units: ΔF/F₀
- Values represent fractional fluorescence change (e.g., 0.25 = 25% increase)
- Typical range: ~0.05 to ~2.0+ depending on indicator and calcium influx
- Standard, publishable units comparable across studies

### Biological Interpretation
- **Higher amplitude** → Larger calcium influx or stronger neuronal activation
- **Lower amplitude** → Smaller calcium signals
- Useful for detecting changes in neuronal excitability or calcium channel function
- Can indicate the "strength" of neuronal responses

In [ ]:
# VIEW PEAK AMPLITUDE DATA
# -------------------------
# This table shows the height of each calcium transient in ΔF/F₀ units.

if orb.peak_amplitude_data is not None:
    print("Peak Amplitude Data (ΔF/F₀ units)")
    print("="*50)
    print(f"\nFirst 10 rows:")
    display(orb.peak_amplitude_data.head(10))
    
    # Quick summary statistics
    print(f"\n📊 Summary Statistics (ΔF/F₀):")
    print(f"   Mean amplitude: {orb.peak_amplitude_data['Peak Amplitudes'].mean():.4f}")
    print(f"   Median amplitude: {orb.peak_amplitude_data['Peak Amplitudes'].median():.4f}")
    print(f"   Range: {orb.peak_amplitude_data['Peak Amplitudes'].min():.4f} - {orb.peak_amplitude_data['Peak Amplitudes'].max():.4f}")
    
    # Interpretation help
    print(f"\n💡 Interpretation:")
    print(f"   0.10 = 10% fluorescence increase")
    print(f"   0.50 = 50% fluorescence increase")
    print(f"   1.00 = 100% fluorescence increase (doubling)")
else:
    print("❌ No peak amplitude data available.")

## 6.7: Inter-Spike Interval Data (Peak-to-Peak Distance)

### What It Measures
**Inter-spike interval (ISI)** = The time between consecutive calcium transient peaks (in frames).

### Biological Interpretation
- **Regular ISIs** (low variability) → Rhythmic/pacemaker-like firing
- **Irregular ISIs** (high variability) → Bursty or random firing patterns
- The coefficient of variation (CV) of ISIs indicates firing regularity:
  - CV < 1: More regular than random
  - CV = 1: Random (Poisson-like) firing  
  - CV > 1: Bursty firing patterns
- Useful for identifying oscillatory activity or changes in firing patterns

In [ ]:
# VIEW INTER-SPIKE INTERVAL DATA
# -------------------------------
# This table shows the time between consecutive peaks for each neuron.

import numpy as np

if orb.peak_to_peak_data is not None:
    print("Inter-Spike Interval Data (frames between consecutive peaks)")
    print("="*50)
    print(f"\nFirst 10 rows:")
    display(orb.peak_to_peak_data.head(10))
    
    # Calculate firing regularity (coefficient of variation)
    print("\n📊 Firing Regularity Analysis:")
    isi_data = orb.peak_to_peak_data.dropna(subset=['Inter-Spike Intervals'])
    
    if len(isi_data) > 0:
        # Calculate CV per neuron
        isi_cv = isi_data.groupby(['Sample', 'Neuron'])['Inter-Spike Intervals'].agg(
            lambda x: x.std() / x.mean() if len(x) > 1 and x.mean() > 0 else np.nan
        ).dropna()
        
        if len(isi_cv) > 0:
            print(f"   Mean ISI: {isi_data['Inter-Spike Intervals'].mean():.2f} frames")
            print(f"   Mean CV across neurons: {isi_cv.mean():.3f}")
            print(f"\n   Interpretation:")
            mean_cv = isi_cv.mean()
            if mean_cv < 0.5:
                print("   → Very regular firing (rhythmic/oscillatory)")
            elif mean_cv < 1.0:
                print("   → Moderately regular firing")
            else:
                print("   → Irregular/bursty firing patterns")
else:
    print("❌ No inter-spike interval data available.")

In [ ]:
# VIEW CLUSTERING DATA
# --------------------
# This table shows the quality of neuron grouping for each sample.

if orb.silhouette_scores_data is not None:
    print("Clustering Quality (Silhouette Scores)")
    print("="*50)
    print(f"\nSilhouette scores by sample:")
    display(orb.silhouette_scores_data)
    
    print("\nInterpretation:")
    print("   • Score > 0.5: Strong clustering (distinct neuron groups)")
    print("   • Score 0.25-0.5: Moderate clustering")
    print("   • Score < 0.25: Weak clustering (overlapping groups)")
else:
    print("❌ No clustering data available.")

---

# Section 7: Correlation & Network Analysis

**Learning objective**: Understand network synchrony and run dedicated correlation analysis.

---

Network synchrony is a key readout in calcium imaging — it tells you whether neurons are firing together (coordinated activity) or independently. This section covers both the correlation metrics from `run_all()` and the dedicated `run_pwc()` method for deeper network analysis.

## 7.1: Pairwise Correlation Data

### What It Measures
**Pairwise correlations** = How similar the activity patterns are between pairs of neurons.

### Types of Correlations
- **Overall** (`df_mn_pwc`) → Average correlation across all neuron pairs
- **Intra-group** (`df_mn_pwc_intra`) → Correlations within the same condition/well
- **Inter-group** (`df_mn_pwc_inter`) → Correlations between different conditions/wells

### Biological Interpretation
- **High correlation** → Neurons fire together; may be part of the same circuit
- **Low correlation** → Neurons fire independently
- Compare intra vs. inter correlations to assess network synchrony between conditions

In [ ]:
# VIEW PAIRWISE CORRELATION DATA
# ------------------------------
# This table shows how synchronized neurons are within and between groups.

print("Pairwise Correlation Analysis")
print("="*50)

if orb.df_mn_pwc is not None:
    print("\nOverall Correlations (all neuron pairs):")
    display(orb.df_mn_pwc)
else:
    print("\n⚠️ No overall correlation data. Did you set group_name in run_all()?")

if orb.df_mn_pwc_intra is not None:
    print("\nIntra-group Correlations (within same condition):")
    display(orb.df_mn_pwc_intra)

if orb.df_mn_pwc_inter is not None:
    print("\nInter-group Correlations (between conditions):")
    display(orb.df_mn_pwc_inter)

## 7.2: Running Dedicated Correlation Analysis with `run_pwc()`

If you want to analyze network correlations **separately** from other metrics (e.g., with different filtering parameters, or without recalculating firing rates), use the dedicated `run_pwc()` method:

**When to use `run_pwc()` instead of relying on `run_all()`:**
- You've already run `run_all()` but want to re-analyze correlations with different settings
- You only care about network synchrony, not individual neuron metrics
- You want to compare correlation patterns across different filtering thresholds

In [ ]:
import importlib
from wizards_staff import pwc
importlib.reload(pwc)
# RUN PAIRWISE CORRELATION ANALYSIS ONLY
# --------------------------------------
# This analyzes how neurons are correlated within and between groups.

orb.run_pwc(
    group_name="Well",     # Which metadata column defines your groups?
    poly=True,             # Use polynomial fitting for smoother trends
    p_th=75,               # Brightness filter (same as run_all)
    size_threshold=20000,  # Size filter (same as run_all)
    show_plots=True        # Show correlation plots
)

print("\nPairwise correlation analysis complete!")


---

## Ready for Statistical Testing?

You've now extracted all the core metrics from your data:
- **Firing rates** (Section 6.1) — how active are your neurons?
- **Rise times & FWHM** (Section 6.2-6.3) — what are the kinetics of calcium events?
- **Clustering** (Section 6.4) — are there functional neuron groups?
- **Network correlations** (Section 7) — how synchronized is neuronal activity?

The next step is to determine whether differences between your experimental groups are **statistically significant**. Section 8 covers:
- Proper aggregation to biological replicates (avoiding pseudoreplication)
- Assumption checking (normality, variance homogeneity)
- Choosing the right statistical test
- Effect size interpretation and power analysis

---

# Section 8: Statistical Analysis of Group Differences

**Learning objective**: Perform rigorous statistical comparisons between experimental groups using proper methods for calcium imaging data.

---

## Why Proper Statistical Analysis Matters

When analyzing calcium imaging data, it's crucial to:

1. **Avoid pseudoreplication**: Individual neurons are *technical replicates* within a sample. Statistical tests should be performed on *sample-level* summaries (biological replicates).

2. **Check assumptions**: Different tests have different requirements (normality, equal variance).

3. **Report effect sizes**: P-values alone don't tell you if an effect is biologically meaningful.

4. **Correct for multiple comparisons**: When running many tests, adjust p-values to control false discoveries.

The `wizards_staff.stats` module handles all of this for you!

In [ ]:
# =============================================================================
# IMPORT STATISTICAL ANALYSIS TOOLS
# =============================================================================

from wizards_staff.stats import (
    # Core functions
    prepare_for_stats,
    StatsConfig,
    
    # Statistical tests
    compare_two_groups,
    compare_multiple_groups,
    
    # Assumption checking
    check_all_assumptions,
    
    # Effect sizes
    cohens_d,
    
    # Power analysis
    calculate_required_n,
    calculate_achieved_power,
    calculate_detectable_effect,
    
    # Visualization
    plot_group_comparison,
    plot_power_curve,
)

print("✅ Statistical analysis tools loaded successfully!")


## 8.1: Prepare Your Data for Statistics

**The most important step!** This aggregates your neuron-level data to sample-level summaries.

### Why is this critical?
- You have ~100 neurons per sample, but maybe only 5 samples per group
- Using neuron-level data would give you n=500 when you really have n=5
- This "pseudoreplication" inflates your statistical power artificially
- Result: False positives and unreproducible findings

In [ ]:
# =============================================================================
# STEP 1: PREPARE YOUR DATA
# =============================================================================
# This step aggregates neuron-level measurements to sample-level summaries.

# Prepare firing rate data for statistical analysis
frpm_stats_df = prepare_for_stats(
    orb,
    metric="frpm",
    aggregate=True,  # Aggregate neurons to sample-level (RECOMMENDED)
    metadata_cols=["Well"]  # Include experimental grouping columns
)

# View the prepared data
print("Sample-level summary statistics:")
print(f"Number of biological replicates: {len(frpm_stats_df)}")
print(f"Groups: {frpm_stats_df['Well'].unique()}")
print()
display(frpm_stats_df.head(10))


In [ ]:
# =============================================================================
# STEP 1b: ADD TREATMENT GROUPS
# =============================================================================
# The statistical functions need a column with treatment groups (e.g., "Control" vs "Drug").
# Map your Well IDs to treatment groups based on your experimental design.

# EXAMPLE: Map well rows to treatment groups
# In this demo plate layout, we split wells into two groups by row letter:
#   - Wells in rows F, G, H = "Control"
#   - Wells in rows I, J, K = "Drug"
# 
# ⚠️ MODIFY THIS MAPPING for your actual experimental design!

def well_to_treatment(well):
    """Convert well ID to treatment group based on row letter."""
    row = well[0].upper()  # First character is the row (e.g., 'F' from 'F02')
    # Split alphabet roughly in half for demo purposes
    if row in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
        return "Control"
    else:  # I, J, K, L, M, N, O, P
        return "Drug"

frpm_stats_df["Treatment"] = frpm_stats_df["Well"].apply(well_to_treatment)

# Verify the treatment mapping - should show exactly 2 groups
print("Treatment groups created:")
print(frpm_stats_df.groupby("Treatment").size())
print(f"\nDataFrame now has columns: {list(frpm_stats_df.columns)}")
print(f"\n✅ Ready for two-group comparison!" if frpm_stats_df["Treatment"].nunique() == 2 else "⚠️ Check your mapping - need exactly 2 groups")

## 8.2: Check Statistical Assumptions

Before running statistical tests, verify your data meets the assumptions. This determines whether to use **parametric** (e.g., t-test) or **non-parametric** (e.g., Mann-Whitney) tests.

In [ ]:
# =============================================================================
# STEP 2: CHECK STATISTICAL ASSUMPTIONS
# =============================================================================

assumptions = check_all_assumptions(
    data=frpm_stats_df,
    group_col="Treatment",  # Your experimental grouping column
    metric_col="mean_Firing Rate Per Min",
    test_type="two_group"
)

# Print the assumption check report
print(assumptions["report"])
print(f"\nRecommended approach: {assumptions['recommendation']}")


## 8.3: Compare Two Groups

Now run the statistical comparison. The `compare_two_groups()` function:
- Chooses the appropriate test based on your assumptions
- Calculates effect size with 95% confidence interval
- Generates a plain-language interpretation

### Key Parameters

| Parameter | Your Choice | Alternatives |
|-----------|-------------|--------------|
| `group_col` | `"Treatment"` | Any column with exactly 2 groups (e.g., `"Genotype"`, `"Condition"`) |
| `metric_col` | `"mean_Firing Rate Per Min"` | Any metric column from your stats dataframe |
| `parametric` | `"auto"` | `True` (force t-test), `False` (force Mann-Whitney) |

💡 **Tip**: If you have more than 2 groups, use `compare_multiple_groups()` instead—see Section 18.

In [ ]:
# =============================================================================
# STEP 3: COMPARE EXPERIMENTAL GROUPS
# =============================================================================

# Configure statistical analyses (optional - uses sensible defaults)
config = StatsConfig(
    alpha=0.05,           # Significance level
    correction_method="fdr_bh",  # False discovery rate correction for multiple tests
    min_samples_per_group=3      # Minimum biological replicates
)

# Compare two groups
# NOTE: Use your experimental grouping column (e.g., "Treatment"), not individual well IDs
result = compare_two_groups(
    data=frpm_stats_df,
    group_col="Treatment",  # Changed from "Well" to use our 2-group column
    metric_col="mean_Firing Rate Per Min",
    parametric="auto",  # Automatically choose based on assumptions
    config=config
)

# Print human-readable summary
print(result.summary)


In [ ]:
# =============================================================================
# VISUALIZE THE COMPARISON
# =============================================================================

# Create a publication-ready comparison plot
fig = plot_group_comparison(
    data=frpm_stats_df,
    group_col="Treatment",  # Changed from "Well" to use our 2-group column
    metric_col="mean_Firing Rate Per Min",
    stats_result=result,
    plot_type="box",
    show_points=True,
    title="Firing Rate by Treatment Group",
    ylabel="Firing Rate (spikes/min)",  # Clean y-axis label
    xlabel="Treatment"
)
plt.show()


## 8.4: Power Analysis

Power analysis helps you understand:
- **Before experiment**: How many samples do you need?
- **After experiment**: What effects could you have detected?

In [ ]:
# =============================================================================
# POWER ANALYSIS
# =============================================================================

# How many samples would I need to detect a medium effect?
required = calculate_required_n(
    effect_size=0.5,  # Medium effect (Cohen's d)
    power=0.80,       # 80% power (conventional)
    alpha=0.05,
    test_type="two_sample_t"
)

print("📊 Sample Size Planning")
print("=" * 60)
print(required["interpretation"])

# What's the minimum effect I could detect with my current samples?
n_per_group = len(frpm_stats_df) // 2  # Approximate

detectable = calculate_detectable_effect(
    n_per_group=n_per_group,
    power=0.80
)

print("\nSensitivity Analysis")
print("=" * 60)
print(detectable["interpretation"])

# Visualize power relationships
fig = plot_power_curve(n_per_group=n_per_group)
plt.show()


---

## Statistical Analysis Quick Reference

### Which Test to Use?

| Situation | Recommended Test |
|-----------|-----------------|
| 2 groups, normal data | Independent t-test |
| 2 groups, non-normal data | Mann-Whitney U |
| 2 paired groups | Paired t-test or Wilcoxon |
| 3+ groups, normal data | One-way ANOVA |
| 3+ groups, non-normal data | Kruskal-Wallis |
| Correlation (normal) | Pearson |
| Correlation (non-normal) | Spearman |

### Effect Size Interpretation

| Cohen's d | Interpretation |
|-----------|---------------|
| < 0.2 | Negligible |
| 0.2 - 0.5 | Small |
| 0.5 - 0.8 | Medium |
| > 0.8 | Large |

### Key Principles

1. **Always aggregate to biological replicates** - neurons within a sample are NOT independent
2. **Check assumptions** before choosing a test
3. **Report effect sizes** alongside p-values
4. **Correct for multiple comparisons** when running many tests

---

# Section 9: Visualizations & Publication Figures

**Learning objective**: Generate and customize plots for your data.

---

Beyond the automatic plots from `run_all()`, you can create custom visualizations for presentations and publications.

## 9.1: Compare Metrics Between Groups

In [ ]:
# This plot shows average firing rates for each condition/well.
from wizards_staff.plotting import plot_metric_by_group

# Simple one-liner to plot FRPM by group
plot_metric_by_group(orb, 
                    metric="frpm", 
                    group_col="Well", 
                    show_plots=True,
                    aggregate=False,
                    show_individual_points=True)


### Plotting Other Metrics

The `plot_metric_by_group()` function works with all calculated metrics:

| Metric | Description | Time Convertible |
|--------|-------------|------------------|
| `frpm` | Firing Rate Per Minute | ❌ |
| `rise_time` | Calcium signal rise times | ✅ → seconds/ms |
| `fwhm` | Full Width at Half Maximum (event duration) | ✅ → seconds/ms |
| `spike_counts` | Number of spikes per neuron | ❌ |
| `roundness` | Cell/organoid shape metric | ❌ |
| `diameter` | Cell/organoid size | ❌ |
| `area` | Cell/organoid area | ❌ |
| `silhouette` | Clustering quality score | ❌ |

**Options:**
- **Plot types**: `"bar"`, `"box"`, `"violin"`
- **Time units**: `"ms"` (milliseconds) or `"s"` (seconds) — auto-converts using frame rate from metadata
- **Palettes**: `"Set2"`, `"Dark2"`, `"tab10"` (categorical) or custom colors list


In [ ]:
# Example: Box plot of FWHM (event duration) in milliseconds
# The function auto-extracts frame rate from metadata to convert frames → time
plot_metric_by_group(orb, 
                    metric="fwhm",
                    group_col="Well", 
                    plot_type="box", 
                    time_unit="ms",
                    aggregate=False,
                    show_individual_points=True)

# Example: Rise time in seconds with a different color palette
# plot_metric_by_group(orb, metric="rise_time", group_col="Well", 
#                      plot_type="violin", time_unit="s", palette="Dark2")

# Example: Custom colors for your groups
# plot_metric_by_group(orb, metric="frpm", group_col="Well",
#                      colors=['#2ecc71', '#e74c3c', '#3498db', '#9b59b6'])

# Available metrics: "frpm", "fwhm", "rise_time", "spike_counts", "roundness", "diameter", "area", "silhouette"
# Available plot types: "bar", "box", "violin"
# Recommended palettes: "Set2", "Dark2", "tab10", "Paired" (categorical), "Blues", "viridis" (sequential)


## 9.2: Using Built-in Plotting Functions

Wizards Staff includes several specialized plotting functions for individual samples:

In [ ]:
# IMPORT PLOTTING FUNCTIONS
# -------------------------
from wizards_staff.plotting import (
    plot_spatial_activity_map,  # Shows where neurons are located
    plot_dff_activity,          # Shows calcium traces over time
    plot_cluster_activity       # Shows neuron groupings
)

print("Plotting functions loaded!")


In [ ]:
# PLOT FOR A SPECIFIC SAMPLE
# --------------------------
# Get the first sample to demonstrate plotting functions.

# Get one sample from the Orb
sample_iterator = orb.shatter()  # This lets us access individual samples
shard = next(sample_iterator)    # Get the first sample

print(f"Working with sample: {shard.sample_name}")


In [ ]:
# SPATIAL ACTIVITY MAP
# --------------------
# This shows where each detected neuron is located in your field of view.
# Each colored blob is one neuron's spatial footprint.

# First, apply filtering to get good neurons
filtered_idx = shard.spatial_filtering(
    p_th=75,               # Brightness threshold
    size_threshold=20000,  # Size threshold
    plot=False,            # Don't show the filtering plot
    silence=True           # Don't print messages
)

print(f"Found {len(filtered_idx)} neurons after filtering")

# Plot the spatial map
plot_spatial_activity_map(
    im_min=shard.get_input('minprojection'),  # Background image
    cnm_A=shard.get_input('cnm_A'),           # Neuron locations
    cnm_idx=filtered_idx,                      # Which neurons to show
    sample_name=shard.sample_name,
    clustering=False,      # Color randomly (set True to color by functional group)
    show_plots=True,
    save_files=False
)


In [ ]:
# SPATIAL ACTIVITY MAP WITH CLUSTERING
# ------------------------------------
# Same as the spatial map above, but colors neurons by their functional cluster.
# Neurons with the same color fire together!

plot_spatial_activity_map(
    im_min=shard.get_input('minprojection'),
    cnm_A=shard.get_input('cnm_A'),
    cnm_idx=filtered_idx,
    sample_name=shard.sample_name,
    clustering=True,                          # Color by cluster!
    dff_dat=shard.get_input('dff_dat'),       # Required for clustering
    show_plots=True,
    save_files=False
)


In [ ]:
# CALCIUM TRACE PLOT
# ------------------
# This shows the activity of each neuron over time.
# Each row is one neuron; brighter colors = higher calcium (more activity).

# Get frame rate from this sample's metadata
sample_frate = int(shard.metadata['Frate'].iloc[0])

plot_dff_activity(
    dff_dat=shard.get_input('dff_dat'),  # Calcium traces
    cnm_idx=filtered_idx,                 # Which neurons to show
    frate=sample_frate,                   # Frame rate from metadata
    sample_name=shard.sample_name,
    max_z=0.6,                            # Color scale maximum
    begin_tp=0,                           # Start frame
    end_tp=1000,                          # End frame (first ~33 sec at 30 fps)
    show_plots=True,
    save_files=False
)


In [ ]:
# CLUSTER ACTIVITY PLOT
# ---------------------
# This groups neurons by their activity patterns and shows each cluster.
# Neurons in the same cluster fire with similar patterns.

plot_cluster_activity(
    dff_dat=shard.get_input('dff_dat'),
    filtered_idx=filtered_idx,
    sample_name=shard.sample_name,
    min_clusters=2,      # Minimum number of groups to try
    max_clusters=10,     # Maximum number of groups to try
    norm=True,           # Normalize activity for comparison
    show_plots=True,
    save_files=False
)


---

# Section 10: Exporting Results & Reports

**Learning objective**: Save your results for further analysis and publication.

---

## 10.1: Automatic Export (If You Used `save_files=True`)

If you ran `orb.run_all()` with `save_files=True`, your results are already saved as CSV files in the output folder.

## 10.2: Manual Export

You can also manually save specific results:

In [ ]:
# SAVE ALL RESULTS TO CSV FILES
# -----------------------------
# This saves your results as spreadsheet-compatible CSV files.

import os

# Create output directory if it doesn't exist
export_folder = "./exported_results/"
os.makedirs(export_folder, exist_ok=True)

# Save all available results
orb.save_results(
    outdir=export_folder,
    result_names=[
        "rise_time_data",        # Rise times
        "fwhm_data",             # Event durations
        "frpm_data",             # Firing rates
        "mask_metrics_data",     # Cell shape metrics (if available)
        "silhouette_scores_data", # Clustering quality
        "df_mn_pwc",             # Overall correlations
        "df_mn_pwc_intra",       # Within-group correlations
        "df_mn_pwc_inter"        # Between-group correlations
    ]
)

print(f"\n Results saved to: {export_folder}")
print("\nFiles created:")
for f in os.listdir(export_folder):
    if f.endswith('.csv'):
        print(f"   • {f}")


In [ ]:
from wizards_staff.stats import quick_report

# Generate a complete HTML report with one line
quick_report(orb, group_col="Treatment", output_path="analysis_report.html")

---

## End of Part 2

You've completed the **Unpaired Group Comparisons** workflow! 🎉

**What you accomplished:**
- Ran the complete analysis pipeline
- Understood firing rate, rise time, FWHM, and network correlation metrics
- Performed rigorous statistical comparisons
- Created publication-ready figures
- Exported your results

---

## What About Paired Data?

If you have **before/after** measurements on the same samples (e.g., baseline → drug treatment), you may want to use **Drug Response Analysis** instead.

➡️ **Continue to Part 3 (Section 11)** for paired baseline-dosing workflows.

➡️ **Or skip to Part 4 (Section 17)** for advanced topics and troubleshooting.

---

# Part 3: Paired Drug Response Analysis

*For experiments where the same samples are imaged before and after treatment.*

This part covers:
- Setting up paired data from two Orbs (baseline + dosing)
- Computing fold changes and percent changes
- Statistical tests for paired comparisons
- Visualizing drug effects

---

# Section 11: When to Use Drug Response Analysis

**Learning objective**: Understand when paired analysis is appropriate and how to organize your data.

---

Drug response analysis is designed for **paired experimental designs** where:
- The **same sample** is imaged **twice** (before and after drug treatment)
- You want to measure **fold change** or **percent change** in activity
- You need to account for **sample-to-sample variability** by using each sample as its own control

### Common Use Cases

| Experimental Design | Example |
|--------------------|---------|
| Acute drug effects | Image organoid → apply TTX → image same organoid |
| Dose-response curves | Baseline → 1μM → 10μM → 100μM (sequential imaging) |
| Washout experiments | Baseline → drug → wash → measure recovery |
| Stimulation studies | Baseline → optogenetic/electrical stim → measure response |

### Key Advantages of Paired Analysis

1. **Increased statistical power** - Each sample serves as its own control
2. **Accounts for baseline variability** - Normalizes away sample-to-sample differences
3. **Biologically intuitive** - "2x increase in firing rate" is clearer than "treated group has mean of 15 vs control mean of 8"

---

# Section 12: Setting Up Paired Data

**Learning objective**: Organize your data for paired analysis.

---

## Data Organization

For drug response analysis, you'll have **two separate Lizard-Wizard runs** (recommended approach):

```
/path/to/baseline_results/     ← Imaging BEFORE drug treatment
├── Sample_A1_pre_dff-dat.npy
├── Sample_A1_pre_cnm-A.npy
├── Sample_A2_pre_dff-dat.npy
├── ...

/path/to/dosing_results/       ← Imaging AFTER drug treatment  
├── Sample_A1_post_dff-dat.npy
├── Sample_A1_post_cnm-A.npy
├── Sample_A2_post_dff-dat.npy
├── ...
```

### Metadata Files

The metadata files for both runs should have matching `Well` values to enable pairing:

**Baseline metadata:**
```csv
Sample,Well,Frate,Treatment
Sample_A1_pre,A1,30,Control
Sample_A2_pre,A2,30,Control
Sample_B1_pre,B1,30,DrugX
Sample_B2_pre,B2,30,DrugX
```

**Dosing metadata:**
```csv
Sample,Well,Frate,Treatment
Sample_A1_post,A1,30,Control
Sample_A2_post,A2,30,Control  
Sample_B1_post,B1,30,DrugX
Sample_B2_post,B2,30,DrugX
```

💡 **Key point**: The `Well` values must match between baseline and dosing for pairing to work!

---

# Section 13: Running the Drug Response Comparison

**Learning objective**: Execute the drug response analysis workflow.

---

In [ ]:
# =============================================================================
# DRUG RESPONSE ANALYSIS: Complete Workflow
# =============================================================================

from wizards_staff import Orb
from wizards_staff.drug_response import compare_baseline_dosing

# Step 1: Set your paths
# -----------------------
# Update these paths to match your data location

BASELINE_FOLDER = "/path/to/baseline_results"   # Pre-drug imaging
DOSING_FOLDER = "/path/to/dosing_results"       # Post-drug imaging
METADATA_BASELINE = "/path/to/metadata_baseline.csv"
METADATA_DOSING = "/path/to/metadata_dosing.csv"  # Or same as baseline if structure matches

# Step 2: Create Orbs for both timepoints
# ----------------------------------------
print("Creating Orbs for baseline and dosing data...")

orb_baseline = Orb(
    results_folder=BASELINE_FOLDER,
    metadata_file_path=METADATA_BASELINE
)

orb_dosing = Orb(
    results_folder=DOSING_FOLDER, 
    metadata_file_path=METADATA_DOSING
)

print(f"Baseline: {orb_baseline.num_shards} samples loaded")
print(f"Dosing: {orb_dosing.num_shards} samples loaded")


In [ ]:
# =============================================================================
# CREATE SYNTHETIC DOSING ORB FOR TESTING
# =============================================================================
import numpy as np

class SyntheticOrb:
    """Minimal orb-like object for testing drug response analysis."""
    def __init__(self, real_orb, multiplier_range=(0.5, 4.0), seed=42):
        np.random.seed(seed)
        self.metadata = real_orb.metadata.copy()
        
        # Copy shards reference (needed for sample pairing)
        self._shards = real_orb._shards
        
        # Copy and scale FRPM data
        if real_orb.frpm_data is not None:
            self._frpm_data = real_orb.frpm_data.copy()
            multipliers = np.random.uniform(*multiplier_range, size=len(self._frpm_data))
            self._frpm_data["Firing Rate Per Min"] = self._frpm_data["Firing Rate Per Min"] * multipliers
        else:
            self._frpm_data = None
            
        # Copy and scale rise time data
        if real_orb.rise_time_data is not None:
            self._rise_time_data = real_orb.rise_time_data.copy()
            multipliers = np.random.uniform(*multiplier_range, size=len(self._rise_time_data))
            self._rise_time_data["Rise Times"] = self._rise_time_data["Rise Times"] * multipliers
        else:
            self._rise_time_data = None
            
        # Copy and scale FWHM data  
        if real_orb.fwhm_data is not None:
            self._fwhm_data = real_orb.fwhm_data.copy()
            multipliers = np.random.uniform(*multiplier_range, size=len(self._fwhm_data))
            self._fwhm_data["FWHM Values"] = self._fwhm_data["FWHM Values"] * multipliers
        else:
            self._fwhm_data = None
    
    @property
    def frpm_data(self):
        return self._frpm_data
    
    @property
    def rise_time_data(self):
        return self._rise_time_data
    
    @property
    def fwhm_data(self):
        return self._fwhm_data

# Create synthetic dosing from your existing analyzed orb
orb_baseline = orb  # Your already-analyzed orb
orb_dosing = SyntheticOrb(orb_baseline, multiplier_range=(0.5, 4.0))

print("✅ Created synthetic dosing orb!")

In [ ]:
# Step 3: Run standard analysis on BOTH timepoints
# --------------------------------------------------
# This computes firing rate, rise time, FWHM, etc. for each orb

print("\nAnalyzing baseline samples...")
orb_baseline.run_all(
    frate=30,                # Adjust to your frame rate
    show_plots=False,        # Suppress plots during batch processing
    save_files=False         # Don't save individual files yet
)

print("\nAnalyzing dosing samples...")
orb_dosing.run_all(
    frate=30,
    show_plots=False,
    save_files=False
)

print("\n✅ Analysis complete for both timepoints!")


In [ ]:
# Step 4: Run drug response analysis
# ------------------------------------
# This pairs samples, computes fold changes, and creates visualizations
from importlib import reload
from wizards_staff import drug_response
reload(drug_response)
from wizards_staff.drug_response import compare_baseline_dosing

results = compare_baseline_dosing(
    baseline_orb=orb_baseline,
    dosing_orb=orb_dosing,
    pair_by="Well",              # Column used to match baseline↔dosing samples
    metrics=["frpm", "rise_time", "fwhm"],  # Which metrics to analyze
    normalization="fold_change", # Options: "fold_change", "percent_change", "delta"
    group_col="Treatment",       # Optional: group results by this column
    aggregate=True,              # Aggregate neuron-level data to sample means
    agg_func="mean",             # Aggregation function
    time_unit="ms",              # Use milliseconds (auto-detects frame_rate from metadata)
    show_plots=True,             # Display visualizations
    save_files=True,             # Save CSVs and plots
    output_dir="./drug_response_outputs"
)

# View summary of paired samples
print("\nSample Pairing Summary:")
display(results["pairs"])


In [ ]:
# Step 5: Explore the results
# -----------------------------

# View firing rate fold changes
print("Firing Rate Drug Response:")
display(results["frpm"].head())

# View summary statistics by group
if "summary" in results:
    print("\nSummary Statistics by Treatment Group:")
    display(results["summary"])


---

# Section 14: Understanding Normalization Methods

**Learning objective**: Choose the right normalization for your analysis.

---

Drug response analysis offers several normalization options:

| Method | Formula | "No Change" Value | Best For |
|--------|---------|-------------------|----------|
| **fold_change** | dosing / baseline | 1.0 | Multiplicative effects (2x, 3x increase) |
| **percent_change** | ((dosing - baseline) / baseline) × 100 | 0% | Percentage effects (+50%, -25%) |
| **delta** | dosing - baseline | 0 | Absolute differences |
| **log2_fold_change** | log₂(dosing / baseline) | 0 | Symmetric up/down visualization |

### Interpreting Fold Change Values

| Fold Change | Interpretation |
|-------------|---------------|
| 2.0 | Activity **doubled** after drug treatment |
| 1.0 | **No change** - drug had no effect |
| 0.5 | Activity **halved** (50% reduction) |
| 0.0 | Complete **elimination** of activity |

In [ ]:
# Alternative: Using percent_change normalization
# -------------------------------------------------

results_pct = compare_baseline_dosing(
    baseline_orb=orb_baseline,
    dosing_orb=orb_dosing,
    pair_by="Well",
    metrics=["frpm"],
    normalization="percent_change",  # Changed from fold_change
    group_col="Treatment",
    show_plots=True,
    save_files=False
)

# A percent_change of +100% = doubled, -50% = halved
print("\nPercent Change Results:")
display(results_pct["frpm"][["pair_id", "baseline_Firing Rate Per Min", 
                              "dosing_Firing Rate Per Min", "percent_change"]])


---

# Section 15: Statistical Tests for Paired Data

**Learning objective**: Test whether drug effects are statistically significant.

---

After computing fold changes, you may want to test:

1. **One-sample test**: Is the fold change different from 1.0 (no change)?
2. **Two-sample test**: Is the fold change different between treatment groups?

In [ ]:
# Statistical tests on drug response data
# -----------------------------------------

from scipy import stats as sp_stats
import numpy as np

# Get fold change values
frpm_df = results["frpm"]

# TEST 1: One-sample t-test - Is overall fold change different from 1.0?
# ----------------------------------------------------------------------
fold_changes = frpm_df["fold_change"].dropna()
t_stat, p_value = sp_stats.ttest_1samp(fold_changes, popmean=1.0)

print("One-Sample T-Test: Is drug effect different from no change (fold=1.0)?")
print(f"   Mean fold change: {fold_changes.mean():.3f}")
print(f"   t-statistic: {t_stat:.3f}")
print(f"   p-value: {p_value:.4f}")
if p_value < 0.05:
    direction = "increased" if fold_changes.mean() > 1 else "decreased"
    print(f"   ✅ Significant! Drug treatment {direction} activity.")
else:
    print(f"   ⚠️ No significant drug effect detected.")

# TEST 2: Compare drug effect between treatment groups (if applicable)
# ---------------------------------------------------------------------
if "Treatment" in frpm_df.columns:
    groups = frpm_df["Treatment"].unique()
    if len(groups) == 2:
        group1_fc = frpm_df[frpm_df["Treatment"] == groups[0]]["fold_change"].dropna()
        group2_fc = frpm_df[frpm_df["Treatment"] == groups[1]]["fold_change"].dropna()
        
        t_stat2, p_value2 = sp_stats.ttest_ind(group1_fc, group2_fc)
        
        print(f"\nTwo-Sample T-Test: Comparing {groups[0]} vs {groups[1]}")
        print(f"   {groups[0]} mean fold change: {group1_fc.mean():.3f} (n={len(group1_fc)})")
        print(f"   {groups[1]} mean fold change: {group2_fc.mean():.3f} (n={len(group2_fc)})")
        print(f"   t-statistic: {t_stat2:.3f}")
        print(f"   p-value: {p_value2:.4f}")
        if p_value2 < 0.05:
            print(f"   ✅ Significant difference between groups!")
        else:
            print(f"   ⚠️ No significant difference between groups.")


---

# Section 16: Visualizing Drug Response

**Learning objective**: Create publication-ready visualizations for paired data.

---

The drug response analysis automatically generates three types of plots:

1. **Paired Line Plot** - Shows each sample's trajectory from baseline to dosing
2. **Fold Change Distribution** - Violin/box plot of normalized values with reference line
3. **Baseline vs Dosing Scatter** - X-Y plot with identity line (y=x)

## Custom Visualizations

In [ ]:
# Custom plotting with individual functions
# -------------------------------------------

from wizards_staff.plotting import (
    plot_paired_lines,
    plot_fold_change_distribution,
    plot_baseline_vs_dosing_scatter
)

# Get the FRPM results DataFrame
frpm_df = results["frpm"]

# Custom paired line plot with different colors
plot_paired_lines(
    data=frpm_df,
    metric="frpm",
    baseline_col="baseline_Firing Rate Per Min",
    dosing_col="dosing_Firing Rate Per Min",
    group_col="Treatment",
    title="Firing Rate: Before vs After Drug Treatment",
    palette="Dark2",          # Different color palette
    figsize=(10, 7),
    line_alpha=0.8,
    marker_size=100,
    show_plots=True,
    save_files=False
)


## Drug Response Analysis Summary

### Output Files Generated

When `save_files=True`, the following outputs are created:

| File | Description |
|------|-------------|
| `drug-response-frpm.csv` | Firing rate baseline, dosing, and fold changes |
| `drug-response-rise_time.csv` | Rise time baseline, dosing, and fold changes |
| `drug-response-fwhm.csv` | FWHM baseline, dosing, and fold changes |
| `drug-response-summary.csv` | Summary statistics per group |
| `drug_response_*_paired_lines.png` | Paired line plots |
| `drug_response_*_distribution.png` | Fold change distribution plots |
| `drug_response_*_scatter.png` | Baseline vs dosing scatter plots |


---

# Part 4: Advanced Topics & Reference

*Optional — use these sections if you need more control or encounter issues.*

The sections below cover:
- Processing individual samples with custom parameters
- Additional statistical scenarios (ANOVA, correlations, outliers)
- Troubleshooting common problems

---

# Section 17: Processing Samples Individually

**Learning objective**: Customize analysis for specific samples using the `shatter()` method.

---

## When to Use Individual Processing

Sometimes you need more control than `run_all()` provides:
- Different parameters for different samples
- Detailed inspection of specific samples
- Custom analysis steps not included in `run_all()`

## The `shatter()` Method

The Orb can be "shattered" into individual **Shards**, where each Shard represents one sample:

In [ ]:
# PROCESS SAMPLES ONE AT A TIME
# -----------------------------
# This demonstrates how to access and analyze individual samples.

from scipy.stats import zscore  # For spike detection
import numpy as np

# Loop through each sample (Shard) in the Orb
for shard in orb.shatter():
    print(f"\n{'='*60}")
    print(f"Processing: {shard.sample_name}")
    print(f"{'='*60}")
    
    # Show available files for this sample
    print("\nAvailable data:")
    for item_name in shard.files.keys():
        print(f"   • {item_name}")
    
    # Skip to next sample after showing info (remove 'break' to process all)
    break

print("\n Remove the 'break' statement above to process all samples")


## Example: Custom Analysis for One Sample


In [ ]:
# CUSTOM ANALYSIS EXAMPLE
# -----------------------
# This shows the step-by-step analysis that run_all() performs automatically.

from scipy.stats import zscore

# Get one sample
shard = next(orb.shatter())
print(f"Analyzing: {shard.sample_name}\n")

# Step 1: Filter neurons (remove noise and artifacts)
filtered_idx = shard.spatial_filtering(
    p_th=75,
    size_threshold=20000,
    plot=True,  # Show the filtering result
    silence=False
)
print(f"\nStep 1: {len(filtered_idx)} neurons passed filtering")

# Step 2: Convert ΔF/F₀ to calcium signals and detect spikes
calcium_signals, spike_events = shard.convert_f_to_cs(p=2)
print(f"\nStep 2: Converted to calcium signals")
print(f"   Shape: {calcium_signals.shape} (neurons × time points)")

# Step 3: Z-score the spike events for threshold detection
zscored_spikes = zscore(np.copy(spike_events), axis=1)
print(f"\nStep 3: Z-scored spike events")

# Step 4: Calculate metrics for filtered neurons only
rise_times, rise_positions = shard.calc_rise_tm(
    calcium_signals[filtered_idx],
    zscored_spikes[filtered_idx],
    zscore_threshold=3
)
print(f"\nStep 4: Calculated rise times")

# Count how many events were detected
total_events = sum(len(times) for times in rise_times.values())
neurons_with_events = sum(1 for times in rise_times.values() if len(times) > 0)
print(f"   Found {total_events} events in {neurons_with_events} neurons")


## Accessing Raw Data

You can access any data file for a sample using `shard.get_input()`:


In [ ]:
# ACCESS RAW DATA FOR A SAMPLE
# ----------------------------

shard = next(orb.shatter())

# Get the ΔF/F₀ data (calcium traces)
dff_data = shard.get_input('dff_dat')
print(f" ΔF/F₀ data shape: {dff_data.shape}")
print(f"   Rows = neurons, Columns = time points")

# Get the spatial footprints
cnm_A = shard.get_input('cnm_A')
print(f"\nSpatial footprints shape: {cnm_A.shape}")

# Get accepted component indices
cnm_idx = shard.get_input('cnm_idx')
print(f"\n✓ Accepted components: {len(cnm_idx)} neurons")


---

# Section 18: Additional Statistical Scenarios

**Learning objective**: Handle specialized statistical situations.

---

The following cells demonstrate statistical approaches for common experimental designs.

## Scenario A: Comparing More Than Two Groups

When you have 3 or more experimental conditions (e.g., multiple drug concentrations, genotypes, or timepoints), use `compare_multiple_groups()`.

In [ ]:
# =============================================================================
# SCENARIO A: COMPARING 3+ GROUPS
# =============================================================================
# If you have more than two experimental conditions, use ANOVA-like tests

from wizards_staff.stats import compare_multiple_groups

# Prepare your data (adjust column names to match your metadata)
# Example: comparing firing rates across multiple timepoints
if 'Well' in frpm_stats_df.columns:
    n_groups = frpm_stats_df['Well'].nunique()
    
    if n_groups > 2:
        result_multi = compare_multiple_groups(
            data=frpm_stats_df,
            group_col="Well",  # Change to your grouping column
            metric_col="mean_Firing Rate Per Min",
            post_hoc=True  # Perform pairwise comparisons
        )
        
        print(result_multi.summary)
        
        # View post-hoc pairwise comparisons
        if result_multi.post_hoc_table is not None:
            print("\n📊 Post-hoc Pairwise Comparisons:")
            display(result_multi.post_hoc_table)
    else:
        print(f"Only {n_groups} groups found. Use compare_two_groups() for 2 groups.")


## Scenario B: Correlation Between Metrics

Test whether one metric is related to another (e.g., is firing rate correlated with cell size?).

In [ ]:
# =============================================================================
# SCENARIO B: CORRELATION ANALYSIS
# =============================================================================
# Test relationships between continuous variables

from wizards_staff.stats import test_correlation

# Example: Test if FWHM correlates with firing rate
# First, merge the data
try:
    fwhm_stats_df = prepare_for_stats(orb, metric="fwhm", metadata_cols=["Well"])
    
    # Merge FRPM and FWHM data
    merged_df = frpm_stats_df.merge(
        fwhm_stats_df[['Sample', 'mean_FWHM Values']], 
        on='Sample'
    )
    
    corr_result = test_correlation(
        merged_df,
        x_col="mean_Firing Rate Per Min",
        y_col="mean_FWHM Values",
        method="auto"  # Automatically chooses Pearson or Spearman
    )
    
    print("📊 Correlation: Firing Rate vs Event Duration (FWHM)")
    print(corr_result.summary)
    
    # Scatter plot
    fig, ax = plt.subplots(figsize=(6, 5))
    ax.scatter(merged_df["mean_Firing Rate Per Min"], merged_df["mean_FWHM Values"], alpha=0.7)
    ax.set_xlabel('Mean Firing Rate (events/min)')
    ax.set_ylabel('Mean FWHM (frames)')
    ax.set_title(f'r = {corr_result.statistic:.3f}, p = {corr_result.p_value:.3f}')
    plt.show()
    
except Exception as e:
    print(f"Could not perform correlation analysis: {e}")


## Scenario C: Outlier Detection

Identify potential outliers that might be affecting your results.

In [ ]:
# =============================================================================
# SCENARIO C: OUTLIER DETECTION
# =============================================================================
# Identify samples with unusual values

from wizards_staff.stats import detect_outliers, visualize_outliers

# Detect outliers in firing rate
outlier_result = detect_outliers(
    data=frpm_stats_df,
    metric_col="mean_Firing Rate Per Min",
    method="iqr",  # Interquartile range method (robust)
    group_col="Well"  # Check within each group
)

print("📊 Outlier Detection Results")
print("=" * 60)
print(outlier_result['summary'])
print()
print(f"Recommendation: {outlier_result['recommendation']}")

# If outliers found, show them
if outlier_result['n_outliers'] > 0:
    print("\n⚠️ Outlier samples:")
    outlier_samples = frpm_stats_df.loc[outlier_result['outlier_indices']]
    display(outlier_samples[['Sample', 'Well', 'mean_Firing Rate Per Min']])
    
    # Visualize
    fig = visualize_outliers(frpm_stats_df, "mean_Firing Rate Per Min", outlier_result, "Well")
    plt.show()


---

# Section 19: Troubleshooting

**Learning objective**: Diagnose and fix common problems.

---

## Common Issues and Solutions

### ❌ "Missing columns in metadata"

**Cause**: Your metadata CSV doesn't have the required columns.

**Solution**:
1. Open your metadata file
2. Check that it has columns named exactly: `Sample`, `Well`, `Frate`
3. Column names are case-sensitive!

---

### ❌ "No files found" or warnings about missing files

**Cause**: The `Sample` names in your metadata don't match your file names.

**Solution**:
1. Look at your actual file names (e.g., `MyExperiment_A01_dff-dat.npy`)
2. The `Sample` column should contain: `MyExperiment_A01` (everything before `_dff-dat.npy`)
3. Check for typos, extra spaces, or different capitalization

---

### ❌ Empty plots or "No neurons found"

**Cause**: Filtering is too strict for your data.

**Solution**: Try adjusting the filtering parameters:
```python
orb.run_all(
    p_th=50,              # Lower brightness threshold (was 75)
    size_threshold=50000,  # Higher size limit (was 20000)
    ...
)
```

---

### ❌ Very few spikes detected

**Cause**: Z-score threshold is too high for your signal.

**Solution**: Lower the threshold:
```python
orb.run_all(
    zscore_threshold=2,   # More sensitive (was 3)
    ...
)
```

---

### ❌ Too many spikes (noise being detected)

**Cause**: Z-score threshold is too low.

**Solution**: Increase the threshold:
```python
orb.run_all(
    zscore_threshold=4,   # Less sensitive (was 3)
    ...
)
```

---

### ❌ Wrong time scale in results

**Cause**: The `Frate` value in your metadata doesn't match your actual recording frame rate.

**Solution**: Check your microscope settings and update the `Frate` column in your metadata CSV.

---

### ❌ Correlation analysis not running

**Cause**: `group_name` not specified or doesn't match a column.

**Solution**: Make sure `group_name` matches a column in your metadata:
```python
orb.run_all(
    group_name="Well",   # Must match a column name exactly
    ...
)
```

## Diagnostic Code

Use these cells to debug issues:

In [ ]:
# CHECK YOUR METADATA
# -------------------
print("Your metadata columns:")
print(f"   {list(orb.metadata.columns)}")

print("\nRequired columns:")
print("   ['Sample', 'Well', 'Frate']")

# Check for required columns
required = ['Sample', 'Well', 'Frate']
missing = [col for col in required if col not in orb.metadata.columns]
if missing:
    print(f"\n❌ Missing columns: {missing}")
else:
    print("\n✅ All required columns present!")


In [ ]:
# CHECK SAMPLE NAME MATCHING
# --------------------------
import os

print("Checking if sample names match files...\n")

# Get sample names from metadata
metadata_samples = set(orb.metadata['Sample'])

# Get sample names from files
files_in_folder = os.listdir(RESULTS_FOLDER) if os.path.exists(RESULTS_FOLDER) else []
file_samples = set()
for f in files_in_folder:
    if '_dff-dat.npy' in f:
        sample_name = f.replace('_dff-dat.npy', '')
        file_samples.add(sample_name)

# Compare
in_both = metadata_samples & file_samples
only_in_metadata = metadata_samples - file_samples
only_in_files = file_samples - metadata_samples

print(f"✅ Matched samples: {len(in_both)}")
if only_in_metadata:
    print(f"\n⚠️ In metadata but no files found: {len(only_in_metadata)}")
    for s in list(only_in_metadata)[:5]:
        print(f"   • {s}")
if only_in_files:
    print(f"\n⚠️ Files exist but not in metadata: {len(only_in_files)}")
    for s in list(only_in_files)[:5]:
        print(f"   • {s}")



Congratulations thats the tutorial

**Youre a wizard Harry 🧙‍♂️✨**


In [ ]:
# =============================================================================
# NOTEBOOK VALIDATION
# =============================================================================
# Run this cell as a final check that everything worked correctly.

validation_results = []

# Check that orb was created and analyzed
try:
    assert orb is not None
    assert hasattr(orb, 'frpm_data')
    validation_results.append("✓ Orb analysis completed")
except:
    validation_results.append("⚠ Orb analysis not run (expected if using demo)")

# Check that statistical results exist
try:
    assert 'result' in dir() and result is not None
    validation_results.append("✓ Statistical analysis completed")
except:
    validation_results.append("⚠ Statistical analysis not run")

# Check that figures were created
import os
if os.path.exists("figure_firing_rate.png"):
    validation_results.append("✓ Publication figures exported")
else:
    validation_results.append("⚠ Figures not saved (run Scenario F cell)")

# Print validation summary
print("=" * 50)
print("NOTEBOOK VALIDATION SUMMARY")
print("=" * 50)
for r in validation_results:
    print(r)
print("=" * 50)
